# <p style="text-align: center;">MIS 382N: Advanced Predictive Modeling</p>
# <p style="text-align: center;">Assignment 5</p>
## <p style="text-align: center;">Total points: 50 </p>
## <p style="text-align: center;">Due: Mon, November 28</p>



Your homework should be written in a **Jupyter notebook**. Please submit **only one** ipynb file from each group, and include the names of all the group members. Also, please make sure your code runs and the graphics (and anything else) are displayed in your notebook before submitting.

# Problem 1 - Random Forest vs Boosting - Regression (15pts)

In this question, we will compare performance of different ensemble methods for regression problems: [Random Forest](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html), [Gradient Boosting Regressor](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingRegressor.html) (GBR), and [AdaBoost](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostRegressor.html#sklearn.ensemble.AdaBoostRegressor). Board game data set from DataQuest will be used (you can download data from Canvas: 'games.csv').

1. (1) Load the data, (2) remove duplicate rows, (3) remove features of type string (object in pandas), and (4) replace missing values by mean of each column. Then, partition data into features (X) and the target label (y) for regression task. We want to predict the *average_rating*. Use [train_test_split](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) to split data into training and testing: test_size=0.33, random_state=42. (1pt)

2. Use a [Random Forest](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html) to predict average_rating. Find the best parameters (including *n_estimators*) using [GridSearchCV](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). Report the accuracy of your model in terms of RMSE. (4pts)

3. Use [Gradient Boosting Regressor](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingRegressor.html) (GBR), and [AdaBoost](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostRegressor.html#sklearn.ensemble.AdaBoostRegressor) for predicting the targets. Again, find the best parameters (including *n_estimators,* and* learning_rate*), and report corresponding RMSE for each algorithm. (8pts)

4. Which model did you expect to be more accurate in predicting the targets? Why? Did your observation match this expectation? (2pts)

In [52]:
import pandas as pd
import numpy as np
from sklearn.model_selection import (train_test_split,GridSearchCV)
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import (RandomForestRegressor,GradientBoostingRegressor,AdaBoostRegressor)

## Part 1

In [54]:
Games = pd.read_csv('games.csv')
# drop duplicates
Games = Games.drop_duplicates()
# exclude columns of type object i.e. strings
Games = Games.select_dtypes(exclude=['object'])
# replace missing values with mean of each column
Games = Games.fillna(Games.mean())
# split df into target and features
Target = Games['average_rating']
Features = Games.drop(['average_rating'], axis=1)
# split into training and testing set
x_train, x_test, y_train, y_test = train_test_split(Features, Target, test_size=0.33, random_state=42)

## Part 2

In [55]:
RFR = RandomForestRegressor()
# create paramater grid for exhaustive search
param_grid = { 'n_estimators' : [2,6,10,14,18], 'max_features' : ['auto', 'sqrt', 'log2'] }
# make model
cv_rfv = GridSearchCV(estimator = RFR, param_grid=param_grid, cv=5)
# fit model that does exhaustive search for best params
cv_rfv.fit(x_train, y_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'n_estimators': [2, 6, 10, 14, 18], 'max_features': ['auto', 'sqrt', 'log2']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [56]:
print "{}\t:\t{}".format("Best parameters", cv_rfv.best_params_)
print "{}\t:\t{}".format("Best score", cv_rfv.best_score_)

Best parameters	:	{'max_features': 'sqrt', 'n_estimators': 18}
Best score	:	0.888595276217


In [58]:
# make model with best parameters
RFR = RandomForestRegressor(n_estimators = 18, max_features = 'sqrt')
model = RFR.fit(x_train, y_train)
pred_y = model.predict(x_test)
test_MSE = mean_squared_error(y_test, pred_y)
print 'RMSE\t:\t', np.sqrt(test_MSE)

RMSE	:	1.01616489896


## Part 3

### Gradient Boosting

In [59]:
GBR = GradientBoostingRegressor()
# create paramater grid for exhaustive search
param_grid_GBR = { 'learning_rate' : [.001,.01,.1,1], 'loss' : ['ls', 'lad', 'huber', 'quantile'], 'n_estimators' : [100,150,200], 
             'max_features' : ['auto', 'sqrt', 'log2']}
# make model
cv_GBR = GridSearchCV(estimator = GBR, param_grid=param_grid_GBR, cv=5)
# fit model that does exhaustive search for best params
cv_GBR.fit(x_train, y_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_split=1e-07,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=100,
             presort='auto', random_state=None, subsample=1.0, verbose=0,
             warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'n_estimators': [100, 150, 200], 'loss': ['ls', 'lad', 'huber', 'quantile'], 'learning_rate': [0.001, 0.01, 0.1, 1], 'max_features': ['auto', 'sqrt', 'log2']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [60]:
print "{}\t:\t{}".format("Best parameters", cv_GBR.best_params_)
print "{}\t:\t{}".format("Best score", cv_GBR.best_score_)

Best parameters	:	{'max_features': 'auto', 'loss': 'ls', 'learning_rate': 0.1, 'n_estimators': 200}
Best score	:	0.890100219341


In [61]:
# make model with best parameters
GBR = GradientBoostingRegressor(max_features='auto', loss='ls', learning_rate=0.1, n_estimators=200)
model = GBR.fit(x_train, y_train)
pred_y = model.predict(x_test)
test_MSE = mean_squared_error(y_test, pred_y)
print 'RMSE\t:\t', np.sqrt(test_MSE)

RMSE	:	1.01565503258


### AdaBoost

In [85]:
ABR = AdaBoostRegressor()
# create paramter grid for exhaustive search
param_grid_ABR = { 'learning_rate' : [.01,.1,1], 'loss' : ['linear', 'exponential', 'square'], 'n_estimators' : [50,100]}
# make model
cv_ABR = GridSearchCV(estimator = ABR, param_grid=param_grid_ABR, cv=5)
# fit model that does exhaustive search for best params
cv_ABR.fit(x_train, y_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=AdaBoostRegressor(base_estimator=None, learning_rate=1.0, loss='linear',
         n_estimators=50, random_state=None),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'n_estimators': [50, 100], 'loss': ['linear', 'exponential', 'square'], 'learning_rate': [0.01, 0.1, 1]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [86]:
print "{}\t:\t{}".format("Best parameters", cv_ABR.best_params_)
print "{}\t:\t{}".format("Best score", cv_ABR.best_score_)

Best parameters	:	{'n_estimators': 100, 'loss': 'exponential', 'learning_rate': 0.1}
Best score	:	0.863043771081


In [87]:
# make model with best parameters
ABR = AdaBoostRegressor(n_estimators = 100, loss = 'exponential', learning_rate=0.1)
model = ABR.fit(x_train, y_train)
pred_y = model.predict(x_test)
test_MSE = mean_squared_error(y_test, pred_y)
print 'RMSE\t:', np.sqrt(test_MSE)

RMSE	: 1.13308986963


## Part 4

I thought going into it that Gradient Boosting Regressor (GBR) would perform the best simply because it's a very powerful model.
Interestingly, I didn't tune my parameters fully and Random Forest slightly outperformed Boosting but I suspect that if I tried more parameters and fully tuned my boosting model that it would outperform random forest

# Problem 2 - Random Forest vs Boosting - Classification (15 pts)
In this question, we will compare performance of different ensemble methods for classification problems: [Random Forest](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html), [Gradient Boosting Decision Tree](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html) (GBDT), and [AdaBoost](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html#sklearn.ensemble.AdaBoostClassifier). [Spam Classification Data](https://archive.ics.uci.edu/ml/datasets/Spambase) of UCI will be used (you can download data from Canvas: 'spam_uci.csv'). Don't worry about column names. The last column represents target label, 1 if spam and zero otherwise.

1. Load the data and partition it into features (X) and the target label (y) for classification task. Then, use [train_test_split](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) to split data into training and testing: test_size=0.33, random_state=42. (1pt)

2. Use a [Random Forest](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) to classify whether an email is spam. Find the best parameters (including *n_estimators* and *criterion*) using [GridSearchCV](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). Report your testing accuracy ([accuracy_score](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html)) and [roc_auc_score](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html#sklearn.metrics.roc_auc_score). You will need [predict_proba](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html#sklearn.ensemble.RandomForestClassifier.predict_proba) for roc_auc_score. (4pts)

3. Use [Gradient Boosting Decision Tree](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html) (GBDT), and [AdaBoost](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html#sklearn.ensemble.AdaBoostClassifier) for the spam classification problem. Again, find the best parameters (including *n_estimators, learning_rate,* and *max_depth (GBDT only)*), and report corresponding accuracy_score and roc_auc_score on the test data for each algorithm. (8pts)

4. Point out one advantage and one disadvantage of Random Forest compared to GBDT (2pts)

In [105]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.model_selection import (train_test_split,GridSearchCV)
from sklearn.metrics import (accuracy_score,roc_auc_score)
from sklearn.ensemble import (RandomForestClassifier,GradientBoostingClassifier,AdaBoostClassifier)
from sklearn.metrics import roc_curve

## Part 1

In [101]:
Spam_uci = pd.read_csv('spam_uci.csv')
# split df into target and features
Target = Spam_uci[[-1]]
Features = Spam_uci.drop(['57'], axis=1)
# split into training and testing set
x_train, x_test, y_train, y_test = train_test_split(Features, Target, test_size=0.33, random_state=42)
x_train = x_train.values
x_test = x_test.values
y_train = y_train.values
# reshape for gridsearch cv's expected data shape
c, r = y_train.shape
y_train = y_train.reshape(c,)
y_test = y_test.values

## Part 2

In [102]:
RFC = RandomForestClassifier()
# create paramter grid for exhaustive search
param_grid = { 'n_estimators' : [10,15,17,19, 21], 'criterion' : ['gini', 'entropy'] }

def hypertuning(estimator, param_grid, x_train, y_train):
    '''this function takes as input an estimator, parameter grid to tune, and the features and target variable
    
        it return the best score and best parameters of this estimaor from given possible params'''
    
    cv_estimator = GridSearchCV(estimator = estimator, param_grid=param_grid, cv=5)
    cv_estimator.fit(x_train, y_train)
    return cv_estimator.best_score_, cv_estimator.best_params_

hypertuning(RFC, param_grid, x_train, y_train)

(0.9993510707332901, {'criterion': 'gini', 'n_estimators': 17})

### Using old function to get accuracy score and roc_auc score

In [106]:
RFC = RandomForestClassifier(criterion = 'gini', n_estimators = 19)
model = RFC.fit(x_train, y_train)
pred_y = model.predict(x_test)
def BayesianClassifier(modeltype, train_X, train_y, test_X):
    
    '''takes bayesian models of Linear/Quadratic Discriminant Analysis, Gaussian Naiive Bayes
    and return an array of predicted probabilities as an array
    
        Note: data must be in an array   '''
    
    fitted_model = modeltype.fit(train_X, train_y)
    predicted_y = modeltype.predict(test_X)
    predicted_probabilities = modeltype.predict_proba(test_X)
    return predicted_probabilities

def RocCurve(predprobs, actual_y):
    '''predicted probabilities should be in form of an array
    
    function returns the fpr, tpr, threshold and the AUC of the ROC Curve as the score'''
    
    pos_probs = []
    for i in predprobs:
        pos_probs.append(i[1])
    fpr, tpr, threshold = roc_curve(actual_y, pos_probs)
    score = roc_auc_score(actual_y, pos_probs)
    return fpr, tpr, threshold, score

In [107]:
predictedprobsRFC = BayesianClassifier(RFC, x_train, y_train, x_test)
score = RocCurve(predictedprobsRFC, y_test)[3]
print "AUC_score\t:\t", score
ACC = accuracy_score(y_test, pred_y) 
print "accuracy score\t:\t", ACC

AUC_score	:	1.0
accuracy score	:	0.999341672153


## Part 3

### Gradient Boosted Decision Tree

In [108]:
GBC = GradientBoostingClassifier()
param_grid = { 'n_estimators' : [200, 250, 300], 'learning_rate' : [.0001, .001, .01], 'max_depth' : [1,3,5] }
hypertuning(GBC, param_grid, x_train, y_train)

(1.0, {'learning_rate': 0.001, 'max_depth': 1, 'n_estimators': 250})

In [109]:
GBC = GradientBoostingClassifier(learning_rate = .001, max_depth =1, n_estimators = 250)
model = GBC.fit(x_train, y_train)
pred_y = model.predict(x_test)

predictedprobsGBC = BayesianClassifier(GBC, x_train, y_train, x_test)
score = RocCurve(predictedprobsGBC, y_test)[3]
print "AUC_score\t:\t", score
ACC = accuracy_score(y_test, pred_y) 
print "accuracy score\t:\t", ACC

AUC_score	:	0.998871331828
accuracy score	:	0.998683344305


### AdaBoost

In [110]:
ABC = AdaBoostClassifier()
param_grid = { 'n_estimators' : [25,50, 75], 'learning_rate' : [.001, .01, .1]}
hypertuning(ABC, param_grid, x_train, y_train)

(1.0, {'learning_rate': 0.001, 'n_estimators': 25})

In [111]:
ABC = AdaBoostClassifier(learning_rate = .001, n_estimators = 25)
model = ABC.fit(x_train, y_train)
pred_y = model.predict(x_test)

predictedprobsABC = BayesianClassifier(ABC, x_train, y_train, x_test)
score = RocCurve(predictedprobsABC, y_test)[3]
print "AUC_score\t:\t", score
ACC = accuracy_score(y_test, pred_y) 
print "accuracy score\t:\t", ACC

AUC_score	:	0.998871331828
accuracy score	:	0.998683344305


## Part 4

Random forest is better for scaling up very large dataset for distributed cimputing for its ease of parallelization.
Boosting in general usually outperforms random forest, but you have more parameters you need to hypertune.

# Question 3 - Matrix Factorization for Rating Prediction (20pts)

The movielens dataset contains 1 million movie ratings from several thousand users. We will be using *k*-rank matrix factorization to estimate this dataset as the product $X=UV^T$, where *U* and *V* have only $k$ columns.

1) You can download the movielens 1M dataset from https://datahub.io/dataset/movielens, but for this problem use the data available on Canvas. It has been split into training and test sets, and converted to matrix format where the rows correspond to users and the columns to movies. Note that most of the entries are NaNs, indicating that these ratings are missing. An extra file, lens1m_361M_titles.csv, has been added so you can check out specific movies if you're curious.

2) Scikit-learn is a little behind for recommender systems, and doesn't have any method to factorize matrices with missing data. Which means you get to code it! Slide 22 of the 'apa large scale learning' lecture notes has the equations for stochastic gradient descent on *U* and *V*. You will have to:
* Set up initial guesses for the *U* and *V* matrices. I suggest small random values.
* Find a suitable learning rate for the descent. A learning rate that is too large will probably blow up, like in HW3 problem 1.
* Come up with a stopping policy
* Code the descent algorithm (5 pts)

3) Using your SGD algorithm, apply 2-rank matrix factorization on the filled training matrix. Calculate the RMSE of this model on the training data and on the test data (separately). The optimal score on the training data is around .86 RMSE; your version of gradient descent must go at least below .91 RMSE. (5 pts)

4) You should notice some overfitting. Because matrix factorization learns separate scores for each movie, a movie with very few reviews may be easily overfit. You may want to only predict ratings when you have enough information to reach a good conclusion. Recalculate the RMSE on the test data, specifically for movies with at least 50 reviews (don't retrain the models). Also report the percent of movies that are still included (after cutting those with < 50 reviews), and the percent of test ratings that are still included. (5 pts)

5) Repeat steps 3 and 4 with 5-rank factorization. Display training and test RMSE. (5 pts)

Hints:  
The numpy function *nanmean* is helpful for RMSE calculation.  
The descent algorithm will probably run for at least several minutes.

In [1]:
import numpy as np
import pandas as pd
import random
titles = pd.read_csv('lens1m_361M_titles.csv')
test_X = np.load('lens1m_361M_test.npy')
train_X = np.load('lens1m_361M_train.npy')

## Code the descent algorithm

### Learning Rate: 
Several learning rates (alphas) were tested, and a value of 0.001 was found to be suitable.

### Stopping Policy: 
If the sum of squared errors does not change by more than 0.1% between epochs, we will assume that the algorithm has converged and it will be terminated. This resulted in a training RMSE less than 0.91.

In [18]:
def matrix_factorization(X, U, V, K, epochs=100, alpha=0.001):
    # Transpose the V matrix
    V = V.T
    
    # Initialize e_prev to 0.01 so you are not dividing by 0 on the first iteration
    e_prev = 0.01
    
    # Iterate through each epoch until convergence criteria is met
    for epoch in xrange(epochs):
        
        # Initialize error term for each epoch
        e = 0
        
        # Iterate through each row
        for i in xrange(len(X)):
            # Iterate through each column
            for j in xrange(len(X[i])):
                
                # Only calculate when the cell in the X-matrix is a number greater than 0
                if X[i][j] > 0:
                    eij = X[i][j] - np.dot(U[i,:], V[:,j])
                    
                    # Add squared error to the total error term
                    e = e + eij**2
                    
                    for k in xrange(K):
                        dLdU = -2*eij*V[k][j]
                        dLdV = -2*eij*U[i][k]
                        U[i][k] = U[i][k] - alpha*dLdU
                        V[k][j] = V[k][j] - alpha*dLdV
        
        # Calculate how much the total squared error has changed between epochs
        e_change = abs(e - e_prev)
        # Convert change to percentage
        perc_change = e_change/e_prev
        e_prev = e
        
        # If percent change in error terms between epochs is less than 0.5%, terminate algorithm
        if perc_change < 0.001:
            break
    
    return U, V.T

## 2-Rank Matrix Factorization

### Set up initial guesses for the U and V matrices

In [39]:
# Set n to be number of rows in X
n = len(train_X)
# Set m to be number of columns in X
m = len(train_X[0])

In [40]:
# 2-rank matrix factorization: r = 2
r = 2
# Initialize U and V matrices
U_init = np.random.random((n,r))
V_init = np.random.random((m,r))

### Apply 2-rank matrix factorization on the filled training matrix

In [41]:
# Obtain U and V matrices for training set
U, V = matrix_factorization(train_X, U_init, V_init, r)

# Take dot product of U and V.T to get predictions for the training set
X_pred = np.dot(U, V.T)

### Calculate RMSE of this model on the training data

In [42]:
# Initialize SSE and count terms
sse_train = 0
count = 0
for i in xrange(len(train_X)):
    for j in xrange(len(train_X[i])):
        # Only calculate when the cell in the training matrix is a number greater than 0
        if train_X[i][j] > 0:
            sse_train += (X_predTrain[i][j] - train_X[i][j])**2
            count += 1

# Calculate MSE            
mse_train = sse_train/count
# Calculate RMSE
rmse_train = np.sqrt(mse_train)

### Calculate RMSE of this model on the test data

In [79]:
# Initialize SSE and count terms
sse_test = 0
count = 0
for i in xrange(len(test_X)):
    for j in xrange(len(test_X[i])):
        # Only calculate when the cell in the test matrix is a number greater than 0
        if test_X[i][j] > 0:
            sse_test += (X_pred[i][j] - test_X[i][j])**2
            count += 1

# Calculate MSE            
mse_test = sse_test/count
# Calculate RMSE
rmse_test = np.sqrt(mse_test)

### Results

In [80]:
print '2-RANK MATRIX FACTORIZATION'
print 'Training RMSE\t:\t{}'.format(rmse_train)
print 'Test RMSE\t:\t{}'.format(rmse_test)

2-RANK MATRIX FACTORIZATION
Training RMSE	:	0.90415087504
Test RMSE	:	0.922995105071


### Calculate RMSE on test data for only movies with at least 50 reviews

In [98]:
# Tranpose predicted matrices to be able to iterate through columns
testT_X = test_X.T
X_predT = X_pred.T

sse_test50 = 0
count = 0
movies_included = 0
for i in xrange(len(testT_X)):
    num_movies = np.count_nonzero(~np.isnan(testT_X[i]))
    if num_movies >= 50:
        movies_included += 1
        for j in xrange(len(testT_X[i])):
            if testT_X[i][j] > 0:
                sse_test50 += (X_predT[i][j] - testT_X[i][j])**2
                count += 1

mse_test50 = sse_test50/count
rmse_test50 = np.sqrt(mse_test50)

In [99]:
print 'RMSE\t:\t{}'.format(rmse_test50)
print
print 'Number of Movies Included\t:\t{}'.format(movies_included)
print 'Total Number of Movies\t\t:\t{}'.format(len(test_X[0]))
print 'Percent of Movies Included\t:\t{}%'.format(round(100*float(movies_included)/len(test_X[0]),2))

RMSE	:	0.910592729803

Number of Movies Included	:	1217
Total Number of Movies		:	3952
Percent of Movies Included	:	30.79%


## 5-Rank Matrix Factorization

### Set up initial guesses for the U and V matrices

In [47]:
# 5-rank matrix factorization: r = 5
r = 5
# Initialize U and V matrices
U_init5 = np.random.random((n,r))
V_init5 = np.random.random((m,r))

### Apply 5-rank matrix factorization on the filled training matrix

In [48]:
# Obtain U and V matrices for training set
U_5, V_5 = matrix_factorization(train_X, U_init5, V_init5, r)

# Take dot product of U and V.T to get predictions for the training set
X_pred5 = np.dot(U_5, V_5.T)

### Calculate RMSE of this model on the training data

In [49]:
# Initialize SSE and count terms
sse_train5 = 0
count = 0
for i in xrange(len(train_X)):
    for j in xrange(len(train_X[i])):
        # Only calculate when the cell in the training matrix is a number greater than 0
        if train_X[i][j] > 0:
            sse_train5 += (X_predTrain5[i][j] - train_X[i][j])**2
            count += 1

# Calculate MSE            
mse_train5 = sse_train5/count
# Calculate RMSE
rmse_train5 = np.sqrt(mse_train5)

### Calculate RMSE of this model on the test data

In [50]:
# Initialize SSE and count terms
sse_test5 = 0
count = 0
for i in xrange(len(test_X)):
    for j in xrange(len(test_X[i])):
        # Only calculate when the cell in the test matrix is a number greater than 0
        if test_X[i][j] > 0:
            sse_test5 += (X_pred5[i][j] - test_X[i][j])**2
            count += 1

# Calculate MSE            
mse_test5 = sse_test5/count
# Calculate RMSE
rmse_test5 = np.sqrt(mse_test5)

### Results

In [51]:
print 'Training RMSE\t:\t{}'.format(rmse_train5)
print 'Test RMSE\t:\t{}'.format(rmse_test5)

Training RMSE	:	0.831745091804
Test RMSE	:	0.877515924702


### Calculate RMSE on test data for only movies with at least 50 reviews

In [90]:
# Tranpose predicted matrices to be able to iterate through columns
testT_X = test_X.T
X_predT = X_pred5.T

sse_test50 = 0
count = 0
movies_included = 0
for i in xrange(len(testT_X)):
    num_movies = np.count_nonzero(~np.isnan(testT_X[i]))
    if num_movies >= 50:
        movies_included += 1
        for j in xrange(len(testT_X[i])):
            if testT_X[i][j] > 0:
                sse_test50 += (X_predT[i][j] - testT_X[i][j])**2
                count += 1

mse_test50 = sse_test50/count
rmse_test50 = np.sqrt(mse_test50)

In [96]:
print 'RMSE\t:\t{}'.format(rmse_test50)
print
print 'Number of Movies Included\t:\t{}'.format(movies_included)
print 'Total Number of Movies\t\t:\t{}'.format(len(test_X[0]))
print 'Percent of Movies Included\t:\t{}%'.format(round(100*float(movies_included)/len(test_X[0]),2))

RMSE	:	0.867321400706

Number of Movies Included	:	1217
Total Number of Movies		:	3952
Percent of Movies Included	:	30.79%
